In [1]:
%%capture
%cd scripts

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%matplotlib notebook

# Imports

In [3]:
from itertools import product
import os

import graph_tool.all as gt
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import hypergeom, pearsonr
import seaborn as sns
from sklearn.cluster import KMeans

from functions import *


# Graph-Tool compatibility
plt.switch_backend('cairo')

# Style
sns.set_theme(context='talk', style='white', palette='Set2')
plt.rcParams.update({
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
    'font.size': 22,
    'axes.titlesize': 'medium',
    'axes.labelsize': 'large',
    'xtick.labelsize': 'medium',
    'ytick.labelsize': 'medium',
    'legend.fontsize': 'medium',
    'legend.title_fontsize': 'medium',
    'figure.titlesize': 'x-large',
})

# Figure transparency
# matplotlib.rcParams['figure.facecolor'] = (1., 0., 0., 0.3)  # Debugging
matplotlib.rcParams['figure.facecolor'] = (1., 0., 0., 0.)


In [4]:
# Integrity check
check_ct_edge_specificity()  # Check for duplicate edges with different attentions

100%|███████████████████████████████████████████████████████████████████| 1022/1022 [00:15<00:00, 64.77it/s]


# Metadata

In [5]:
# Load metadata
meta = get_meta()

# Subject preview
filtered = []
for i, row in meta.iterrows():
    try:
        load_graph_by_id(row['SubID'])
        assert not np.isnan(row['nps_MoodDysCurValue'])  # Has NPS information available
        assert row['BRAAK_AD'] in (6,) and row['CERAD'] in (4,) and row['CDRScore'] in (3,)
    except:
        continue
    filtered.append(f'{row["SubID"]} {row["Ethnicity"]} {row["Sex"]}, {row["Age"]}, BRAAK {row["BRAAK_AD"]}, CERAD {row["CERAD"]}, CDR {row["CDRScore"]}, {row["Dx"]}')
filtered = np.sort(filtered)
for i in range(len(filtered)):
    # print(filtered[i])
    pass


# Attention Stack

In [6]:
fname = './attentions.pkl'
if os.path.isfile(fname):
    # Load data
    with open('./attentions.pkl', 'rb') as f:
        all_data = pickle.load(f)
    attention_stack, all_edges, columns, subject_ids = all_data['data'], all_data['edges'], all_data['heads'], all_data['subject_ids']

else:
    # Parameters
    # Scaled probably shouldn't be used, but better for visualization
    # until results are more even
    columns = get_attention_columns(scaled=False)
    subject_ids = meta['SubID'].to_numpy()

    # Load graphs
    graphs, subject_ids = load_many_graphs(subject_ids, column=columns)
    # graphs = [compute_graph(g) for g in graphs]

    # # Get attentions
    # df = {}
    # for column in get_attention_columns():
    #     attention, _ = compute_edge_summary(graphs, subject_ids=subject_ids)
    #     attention = attention.set_index('Edge')
    #     df[column] = attention.var(axis=1)


    # Set indices to edges and clean
    print('Fixing indices...')
    for i in tqdm(range(len(graphs))):
        graphs[i].index = graphs[i].apply(lambda r: get_edge_string([r['TF'], r['TG']]), axis=1)
        graphs[i] = graphs[i].drop(columns=['TF', 'TG'])
        # Remove duplicates
        graphs[i] = graphs[i][~graphs[i].index.duplicated(keep='first')]

    # Get all unique edges
    print('Getting unique edges...')
    all_edges = np.unique(sum([list(g.index) for g in graphs], []))


    # Standardize index order
    print('Standardizing indices...')
    for i in tqdm(range(len(graphs))):
        # Add missing indices and order based on `all_edges`
        # to_add = [edge for edge in all_edges if edge not in list(graphs[i].index)]  # SLOW
        to_add = list(set(all_edges) - set(graphs[i].index))

        # Empty rows
        new_rows = pd.DataFrame(
            [[np.nan]*len(graphs[i].columns)]*len(to_add),
            columns=graphs[i].columns,
        ).set_index(pd.Series(to_add))
        # Native concat
        graphs[i] = pd.concat([graphs[i], new_rows]).loc[all_edges]

    # Convert to numpy
    graphs = [g.to_numpy() for g in graphs]
    attention_stack = np.stack(graphs, axis=-1)
    # attention_stack.shape = (Edge, Head, Subject)
    # attention_stack.shape = (all_edges, columns, subject_ids)

    # Save all data
    all_data = {'data': attention_stack, 'edges': all_edges, 'heads': columns, 'subject_ids': subject_ids}
    # np.savez('attentions.npz', **all_data)
    with open(fname, 'wb') as f:
        pickle.dump(
            all_data,
            f,
            protocol=pickle.HIGHEST_PROTOCOL,
        )


In [7]:
# Additional useful parameters
self_loops = [split_edge_string(s)[0] == split_edge_string(s)[1] for s in all_edges]
self_loops = np.array(self_loops)
# Remove self loops
all_edges = all_edges[~self_loops]
attention_stack = attention_stack[~self_loops]


In [8]:
# Export example data
# for sid_idx in range(5):
#     sid = subject_ids[sid_idx]
#     data = attention_stack[:, :, sid_idx]
#     df = pd.DataFrame(data, index=all_edges, columns=columns).dropna()
#     df.to_csv(f'{sid}.csv')

# Global Parameters

In [9]:
# Parameters
print(f'\nAvailable attention columns: {get_attention_columns()}')
column_ad = get_attention_columns()[0]
column_scz = get_attention_columns()[2]
column_data = get_attention_columns()[4]
synthetic_nodes_of_interest = ['OPC', 'Micro', 'Oligo']



Available attention columns: ['AD_imp_1', 'AD_imp_2', 'SCZ_imp_1', 'SCZ_imp_2', 'data_imp_1', 'data_imp_2', 'data_imp_3', 'data_imp_4']


# Intra-Contrast Comparisons (Figure 4/6)

In [10]:
# Figure parameters
param = {
    'subjects': ['M31969', 'M20337'],
    'columns': [column_data, column_ad, column_scz],
    'column_names': ['Data-Driven', 'AD-Prior', 'SCZ-Prior'],
    'column_groups': [get_attention_columns()[4:8], get_attention_columns()[:2], get_attention_columns()[2:4]],
    'column_group_names': ['Data Prioritization', 'AD Prioritization', 'SCZ Prioritization'],
    'ancestries': meta.groupby('Ethnicity').count()['SubID'].sort_values().index[::-1].to_list()[:3] + ['all'],
    'contrast': 'c15x',
}

# Generate palette
palette = plt.rcParams['axes.prop_cycle'].by_key()['color']
param['palette'] = {sid: rgba_to_hex(palette[i]) for i, sid in enumerate(param['subjects'])}

# Preview subjects
for sid in param['subjects']:
    row = meta.loc[meta['SubID']==sid].iloc[0]
    # print(f'{row["SubID"]} {row["Ethnicity"]} {row["Sex"]}, {row["Age"]}, BRAAK {row["BRAAK_AD"]}, CERAD {row["CERAD"]}, CDR {row["CDRScore"]}, {row["Dx"]}')


In [11]:
# Subplot layout (doesn't work well with constrained layout)
# NOTE: This cannot be used, as constrained layout has glitches
# (see https://github.com/matplotlib/matplotlib/issues/23290)
# with uneven mosaics
# fig, axs = get_mosaic(shape, figsize=(int((3/2) * shape_array.shape[1]), int((3/2) * shape_array.shape[0])), constrained_layout=False)

# Subfigure layout (longer)
# NOTE: Constrained layout will fail for all
# subplots if a single one is not able to scale.
# Also, sometimes leaving a subfigure blank will
# cause it to fail, especially if on an edge.
# It is VERY finnicky.
# SOLUTION: Save again using `fig.savefig(...)`
# and it will run without warning.  Then, you
# can visually inspect for scaling issues.
# fig, axs = create_subfigure_mosaic(shape_array)
# fig.set_constrained_layout_pads(w_pad=0, h_pad=0, wspace=.4, hspace=.4)  # *_pad is pad for figs (including subfigs), *_space is pad between subplots


In [12]:
# np.unique(meta.loc[meta['SubID'].isin(all_data['subject_ids']), 'Ethnicity'], return_counts=True)

## Main (Figure 4)

In [13]:
# Top part
# shape = """
#     .AAAAAABBBBBBNNNNNNNNNNN
#     .AAAAAABBBBBBNNNNNNNNNNN
#     .AAAAAABBBBBBNNNNNNNNNNN
#     .AAAAAABBBBBBNNNNNNNNNNN
#     .AAAAAABBBBBB...........
#     .AAAAAABBBBBBRRRRRRRRRRR
#     .AAAAAABBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
# """
shape = """
    NNNNNNNNNNN
    NNNNNNNNNNN
    NNNNNNNNNNN
    NNNNNNNNNNN
    RRRRRRRRRRR
    RRRRRRRRRRR
    RRRRRRRRRRR
    RRRRRRRRRRR
    RRRRRRRRRRR
    RRRRRRRRRRR
    RRRRRRRRRRR
"""
fig, axs = create_subfigure_mosaic(shape_array_from_shape(shape))

# Plot all panels
# axs_lab = ['A', 'G']
# print(f'Individual Panels ({", ".join(axs_lab)})')
# plot_graph_comparison_from_sids(param['subjects'], axs={0: axs[axs_lab[0]], 1: axs[axs_lab[1]]}, column=param['columns'][0], vertex_ids=synthetic_nodes_of_interest)
# # Legend
# plot_legend(horizontal=False, loc='center', bbox_to_anchor=(.5, -.1), ax=axs[axs_lab[0]])

# axs_lab = ['B']
# print(f'Data Heatmap ({", ".join(axs_lab)})')
# ax=axs[axs_lab[0]] = plot_heatmap_comparison(
#     **all_data,
#     target_sids=param['subjects'],
#     ax=axs[axs_lab[0]],
#     column_groups=param['column_groups'],
#     column_group_names=param['column_group_names'],
#     palette=param['palette'],
#     random_seed=42,
#     num_edges=40)

axs_lab = (len(param['ancestries']) - 1) * ['None'] + ['N']
# axs_lab = ['K', 'L', 'M', 'N']
print(f'\nEdge Discovery Enrichment ({", ".join(axs_lab)})')
for ancestry, ax in zip(param['ancestries'], [axs[lab] if lab in axs else None for lab in axs_lab]):
    # Filter to ancestry
    anc_data = all_data.copy()
    if ancestry != 'all':
        sub_ids = meta.loc[meta['Ethnicity'] == ancestry, 'SubID'].to_list()
        mask = [sid in sub_ids for sid in anc_data['subject_ids']]
        anc_data['data'] = anc_data['data'][:, :, mask]
        anc_data['subject_ids'] = np.array(anc_data['subject_ids'])[mask]

    # Run
    temp = plot_edge_discovery_enrichment(
        **anc_data,
        column=param['columns'][0],
        range_colors=[rgb_to_float(hex_to_rgb('#7aa457')), rgb_to_float(hex_to_rgb('#a46cb7')), rgb_to_float(hex_to_rgb('#cb6a49'))],
        ax=ax,
        postfix=f'{ancestry}_{param["columns"][0]}',
        gene_max_num=300,
        threshold=95,
        clamp_min=4,
        skip_plot=(ax is None),
        verbose=True)
    if ax is not None:
        ax.set_xlabel(f'High-Scoring Edges ({param["column_names"][0]})')
        ylabel = 'Frequency'
        if ancestry != 'all': ylabel += f' ({ancestry})'
        ax.set_ylabel(ylabel)
    # MANUAL PROCESSING
    # Run the output '../plots/genes_<column>.csv' from above on Metascape as multiple gene list and perform
    # enrichment.  From the all-in-one ZIP file, save the file from Enrichment_GO/GO_membership.csv as '../plots/go_<column>.csv'
    # and rerun.

axs_lab = ['R']
print(f'\nAncestry Enrichment Comparison ({", ".join(axs_lab)})')
postfixes = [f'{ancestry}_{param["columns"][0]}' for ancestry in param['ancestries']]
enrichments = plot_cross_enrichment(postfixes, names=param['ancestries'], ax=axs[axs_lab[0]], excluded_subgroups=['all'])

# Place labels
offset = plot_labels(axs, shape=shape)

# Save figure
print('\nSaving Figure...')
fig.savefig(f'../plots/figure_4_main.pdf', bbox_inches='tight', pad_inches=1, format='pdf', transparent=True, backend='cairo')



Edge Discovery Enrichment (None, None, None, N)
Prioritization Ranges [[14.0, 22.0], [22.0, 49.0], [49.0, 680.0]]
Filtered 165669 edges of 204507 total from histogram
Prioritization Ranges [[9.0, 13.0], [13.0, 25.0], [25.0, 104.0]]
Filtered 52826 edges of 61429 total from histogram
Prioritization Ranges [[8.0, 11.0], [11.0, 21.0], [21.0, 88.0]]
Filtered 46979 edges of 53899 total from histogram
Prioritization Ranges [[15.0, 23.0], [23.0, 51.0], [51.0, 875.0]]
Filtered 195859 edges of 242486 total from histogram

Ancestry Enrichment Comparison (R)
Index(['AFR_data_imp_1', 'AMR_data_imp_1', 'EUR_data_imp_1', 'all_data_imp_1'], dtype='object', name='Ancestry')
Index(['EUR', 'AFR', 'AMR', 'all'], dtype='object')
Index(['EUR', 'AFR', 'AMR'], dtype='object')

Saving Figure...


## Supplement (Figure 4)

In [14]:
# Top part
# shape = """
#     .AAAAAABBBBBBNNNNNNNNNNN
#     .AAAAAABBBBBBNNNNNNNNNNN
#     .AAAAAABBBBBBNNNNNNNNNNN
#     .AAAAAABBBBBBNNNNNNNNNNN
#     .AAAAAABBBBBB...........
#     .AAAAAABBBBBBRRRRRRRRRRR
#     .AAAAAABBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
#     .GGGGGGBBBBBBRRRRRRRRRRR
# """
shape = """
    KKKKKKKKKKKLLLLLLLLLLLMMMMMMMMMMM
    KKKKKKKKKKKLLLLLLLLLLLMMMMMMMMMMM
    KKKKKKKKKKKLLLLLLLLLLLMMMMMMMMMMM
    KKKKKKKKKKKLLLLLLLLLLLMMMMMMMMMMM
    NNNNNNNNNNNOOOOOOOOOOOPPPPPPPPPPP
    NNNNNNNNNNNOOOOOOOOOOOPPPPPPPPPPP
    NNNNNNNNNNNOOOOOOOOOOOPPPPPPPPPPP
    NNNNNNNNNNNOOOOOOOOOOOPPPPPPPPPPP
    QQQQQQQQQQQRRRRRRRRRRRSSSSSSSSSSS
    QQQQQQQQQQQRRRRRRRRRRRSSSSSSSSSSS
    QQQQQQQQQQQRRRRRRRRRRRSSSSSSSSSSS
    QQQQQQQQQQQRRRRRRRRRRRSSSSSSSSSSS
    TTTTTTTTTTTUUUUUUUUUUUVVVVVVVVVVV
    TTTTTTTTTTTUUUUUUUUUUUVVVVVVVVVVV
    TTTTTTTTTTTUUUUUUUUUUUVVVVVVVVVVV
    TTTTTTTTTTTUUUUUUUUUUUVVVVVVVVVVV
"""
fig, axs = create_subfigure_mosaic(shape_array_from_shape(shape))

# Plot all panels
axs_lab = ['K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V']
print(f'\nEdge Discovery Enrichment ({", ".join(axs_lab)})')
for (ancestry, column_idx), ax in zip(product(param['ancestries'], range(len(param['columns']))), [axs[lab] if lab in axs else None for lab in axs_lab]):
    # Filter to ancestry
    anc_data = all_data.copy()
    if ancestry != 'all':
        sub_ids = meta.loc[meta['Ethnicity'] == ancestry, 'SubID'].to_list()
        mask = [sid in sub_ids for sid in anc_data['subject_ids']]
        anc_data['data'] = anc_data['data'][:, :, mask]
        anc_data['subject_ids'] = np.array(anc_data['subject_ids'])[mask]

    # Run
    temp = plot_edge_discovery_enrichment(
        **anc_data,
        column=param['columns'][column_idx],
        range_colors=[rgb_to_float(hex_to_rgb('#7aa457')), rgb_to_float(hex_to_rgb('#a46cb7')), rgb_to_float(hex_to_rgb('#cb6a49'))],
        ax=ax,
        postfix=f'{ancestry}_{param["columns"][column_idx]}',
        gene_max_num=300,
        threshold=95,
        skip_plot=(ax is None))
    if ax is not None:
        ax.set_xlabel(f'High-Scoring Edges ({param["column_names"][column_idx]})')
        ylabel = 'Frequency'
        if ancestry != 'all': ylabel += f' ({ancestry})'
        ax.set_ylabel(ylabel)
    # MANUAL PROCESSING
    # Run the output '../plots/genes_<column>.csv' from above on Metascape as multiple gene list and perform
    # enrichment.  From the all-in-one ZIP file, save the file from Enrichment_GO/GO_membership.csv as '../plots/go_<column>.csv'
    # and rerun.

# Place labels
offset = plot_labels(axs, shape=shape)

# Save figure
print('\nSaving Figure...')
fig.savefig(f'../plots/figure_4_supplement.pdf', bbox_inches='tight', pad_inches=1, format='pdf', transparent=True, backend='cairo')



Edge Discovery Enrichment (K, L, M, N, O, P, Q, R, S, T, U, V)

Saving Figure...


## PRS Analyses (Figure 6)

In [15]:
# SCZ
# Data
# AD
# SCZ
shape = """
    SSSSSSSSSSSSSSSUUUUUUUUUUUUUUU
    SSSSSSSSSSSSSSSUUUUUUUUUUUUUUU
    SSSSSSSSSSSSSSSUUUUUUUUUUUUUUU
    SSSSSSSSSSSSSSSUUUUUUUUUUUUUUU
"""
fig, axs = create_subfigure_mosaic(shape_array_from_shape(shape))

# Plot all panels
axs_lab = ['S', 'U']
print(f'\nPRS Analysis ({", ".join(axs_lab)})')
# Takes around an hour for each loop with no subsampling (on first run)
for fname, head_prefix, ylabel, prs_col, ax_idx in zip(
    ('ad_prs_df.csv', 'scz_prs_df.csv'),
    ('_'.join(column_ad.split('_')[:-1]), '_'.join(column_scz.split('_')[:-1])),
    ('AD Importance Score', 'SCZ Importance Score'),
    ('prs_scaled_AD_Bellenguez', 'prs_scaled_SCZ.3.5_MVP'),
    axs_lab
):
    df = pd.read_csv(fname, index_col=0) if os.path.isfile(fname) else None
    covariates = get_genotype_meta()[['SubID', 'imp_sex_score'] + [f'imp_anc_PC{i}' for i in range(1, 7)] + [f'imp_anc_{anc}' for anc in ('AFR', 'AMR', 'EAS', 'EUR')]]
    df, prs_df, axs[ax_idx] = plot_prs_correlation(
        meta, **all_data, ax=axs[ax_idx],
        df=df, num_targets=5, ylabel=ylabel, max_scale=False,
        head_prefix=head_prefix, prs_col=prs_col,
        covariates=covariates, subsample=1)
    if not os.path.isfile(fname): df.to_csv(fname)

# Place labels
offset = plot_labels(axs, shape=shape)

# Save figure
print('\nSaving Figure...')
fig.savefig(f'../plots/figure_6_prs.pdf', bbox_inches='tight', pad_inches=1, format='pdf', transparent=True, backend='cairo')

# CLI
# df



PRS Analysis (S, U)

Saving Figure...


## Extended (Figure 4)

In [16]:
# # Extended
# shape = """
#     EEEEEEFFFFFF
#     EEEEEEFFFFFF
#     EEEEEEFFFFFF
#     BBBBCCCCDDDD
#     BBBBCCCCDDDD
#     BBBBCCCCDDDD
#     BBBBCCCCDDDD
#     BBBBCCCCDDDD
#     HHHHIIIIJJJJ
#     HHHHIIIIJJJJ
#     HHHHIIIIJJJJ
# """
# fig, axs = create_subfigure_mosaic(shape_array_from_shape(shape))

# # Plot all panels
# axs_lab = ['B', 'D']
# print(f'\nCell Type Graph ({", ".join(axs_lab)})')
# pos_dict = {}
# for ax, column, title in zip([axs[lab] for lab in axs_lab], param['columns'], param['column_names']):
#     pos_dict = plot_ct_graph_from_sid(param['subjects'][0], ax=ax, column=column, **pos_dict)
#     ax.set_title(title)

# axs_lab = ['C']
# print(f'\nCell Type Priority ({", ".join(axs_lab)})')
# plot_ct_individual_edge_comparison(param['subjects'][0], **all_data, columns=param['columns'][:2], column_names=param['column_names'][:2], ax=axs[axs_lab[0]])
# axs[axs_lab[0]].set_xlabel(None)

# axs_lab = ['E', 'F']
# print(f'\nAttention Histogram ({", ".join(axs_lab)})')
# ax_ymax = 1
# for ax, column, title in zip([axs[lab] for lab in axs_lab], param['columns'], param['column_names']):
#     plot_attention_histogram(param['subjects'][0], **all_data, column=column, ax=ax)
#     ax.set_xlabel(title)
#     # Record limits
#     ylim = ax.get_ylim()
#     if ylim[1] > ax_ymax: ax_ymax = ylim[1]
# # Set axis limits
# for ax in [axs[lab] for lab in axs_lab]:
#     ax.set_ylim(1, ax_ymax)

# axs_lab = ['H', 'I', 'J']
# print(f'\nEdge Comparisons ({", ".join(axs_lab)})')
# for ax, column, title in zip([axs[lab] for lab in axs_lab], param['column_groups'], param['column_group_names']):
#     ax.clear()
#     plot_edge_comparison_from_sids(param['subjects'], ax=ax, column=column, palette=param['palette'], highlight_outliers=True)
#     ax.set_title(title)

# # Place labels
# offset = plot_labels(axs, shape=shape)

# # Save figure
# print('\nSaving Figure...')
# fig.savefig(f'../plots/figure_4_extended.pdf', bbox_inches='tight', pad_inches=1, format='pdf', transparent=True, backend='cairo')


In [17]:
# # Unused
# shape = """
#     LLLLLLLMMMMMMMNNNNNNN
#     LLLLLLLMMMMMMMNNNNNNN
#     LLLLLLLMMMMMMMNNNNNNN
#     LLLLLLLMMMMMMMNNNNNNN
#     LLLLLLLMMMMMMMNNNNNNN
# """
# fig, axs = create_subfigure_mosaic(shape_array_from_shape(shape))

# # Plot all panels
# axs_lab = ['K']
# print(f'\nModule Analysis ({", ".join(axs_lab)})')
# plot_module_scores_from_sids(param['subjects'], ax=axs[axs_lab[0]], palette=param['palette'], column=param['columns'][0])
# axs[axs_lab[0]].set_title(param['column_names'][0])

# axs_lab = ['L']
# print(f'\nOutcome Icon ({", ".join(axs_lab)})')
# # Manual, so no visualization is needed
# axs[axs_lab[0]].axis('off')

# axs_lab = ['M']
# print(f'\nEdge Discovery ({", ".join(axs_lab)})')
# plot_edge_discovery(**all_data, column=param['columns'][0], ax=axs[axs_lab[0]])
# axs[axs_lab[0]].set_title(param['column_names'][0])

# axs_lab = ['N']
# print(f'\nEnrichment ({", ".join(axs_lab)})')
# # Compute gene list
# fname = f'../plots/genes_{"_".join(param["subjects"])}.csv'
# if not os.path.isfile(fname):
#     df = compute_all_important_genes_from_sids(param['subjects'], **all_data, vertex_ids=synthetic_nodes_of_interest, columns=param['columns'], column_names=param['column_names'])
#     df.to_csv(fname, index=False)
# # MANUAL PROCESSING
# # Run the output from above on Metascape as multiple gene list and perform
# # enrichment.  From the all-in-one ZIP file, save the file from
# # Enrichment_GO/GO_membership.csv as `fname` below
# # Read enrichment
# fname = f'../plots/go_{"_".join(param["subjects"])}.csv'
# if os.path.isfile(fname):
#     plot_enrichment_from_fname(fname, ax=axs[axs_lab[0]])
# axs[axs_lab[0]].set_xlabel(None)

# # Inset axes
# from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# axins = inset_axes(
#     axs[axs_lab[0]],
#     width='20%', height='50%',
#     loc=1,
#     bbox_to_anchor=(-1, -1.3, 1, 1), bbox_transform=axs[axs_lab[0]].transAxes)

# # axs_lab = ['M']
# print(f'\nEdge Discovery ({", ".join(axs_lab)})')
# plot_edge_discovery(**all_data, column=param['columns'][0], ax=axins)  # axs[axs_lab[0]]
# axins.set_title(param['column_names'][0])
# axins.set_xlabel(None)

# # Place labels
# # offset = plot_labels(axs, shape=shape)

# # Save figure
# print('\nSaving Figure...')
# fig.savefig(f'../plots/figure_3_unused.pdf', bbox_inches='tight', pad_inches=.5, format='pdf', transparent=True, backend='cairo')


# Inter-Contrast Comparisons (Figure 4)

In [18]:
# # Combinations
# # TODO: Potentially move each entry to dictionary, so changes in order
# #   are easier to propagate
# contrast_groupings = [
#     # (contrast name, contrast group, attention column, comparison column, target meta column, other target meta column)
#     # for contrast_name, contrast_group, column, comparison, target, target_comparison in contrast_groupings:
#     # TODO: Revise ethnicity prediction
#     ('c15x', 'AD', column_ad, column_data, 'BRAAK_AD', 'Ethnicity'),
#     ('c15x', 'AD', column_data, column_ad, 'BRAAK_AD', 'Ethnicity'),
#     ('c15x', 'AD', column_data, column_ad, 'nps_MoodDysCurValue', 'nps_WtGainCurValue'),
#     # ('c06x', 'AD', column_ad, column_data, 'BRAAK_AD', 'nps_MoodDysCurValue'),  # Eventually SCZ, BP and such
#     # ('c71x', 'MoodDys', column_data, column_ad, 'nps_MoodDysCurValue'),  # Dysphoria
#     # ('c72x', 'DecInt', column_data, column_ad, 'nps_DecIntCurValue'),  # Anhedonia
# ]


### 4B Distribution Comparison

In [19]:
# for contrast_name, _, column, comparison, target, target_comparison in contrast_groupings:
#     # Filter attention stack to contrast
#     contrast = get_contrast(contrast_name)
#     contrast_subject_ids = sum([contrast[group] for group in contrast], [])
#     contrast_mask = [sid in contrast_subject_ids for sid in subject_ids]
#     contrast_subject_ids = np.array(subject_ids)[contrast_mask]
#     contrast_stack = attention_stack[:, :, contrast_mask]

#     # Filter to 1000 most variant edges
#     top_variant_edge_idx = np.nan_to_num(
#         contrast_stack[:, np.argwhere(np.array(columns)==column)[0][0]]).var(axis=1).argsort()[::-1][:1000]
#     contrast_stack = contrast_stack[top_variant_edge_idx]
#     edge_names = all_edges[top_variant_edge_idx]

#     # Correlation df
#     df = pd.DataFrame(
#         contrast_stack[:, np.argwhere(np.array(columns)==column)[0][0]],
#         index=pd.Series(all_edges[top_variant_edge_idx]),
#         columns=contrast_subject_ids).T
#     df = df.join(meta.set_index('SubID')[[target, target_comparison]]).reset_index(drop=True)
#     # Select edge which most cleanly separates `target`
#     # top_distinct_edge_idx = df.drop(target_comparison, axis=1).groupby(target).mean().var(axis=0).argsort()[-1]
#     # Select top 3 most correlating edges
#     edge_name = df.drop(target_comparison, axis=1).corr()[target].abs().drop(target).sort_values(ascending=False)[:3].index.to_numpy()
#     top_distinct_edge_idx = [np.argwhere(df.columns==edge)[0][0] for edge in edge_name]
#     contrast_stack = contrast_stack[top_distinct_edge_idx]

#     # Plot
#     fig, axs = get_mosaic(np.array(sum([[i]*2 for i in range(6)], [])).reshape((2, -1)), scale=5)
#     # axs[0].sharex(axs[1])
#     sns.despine()

#     for i in range(3):
#         # Filter
#         contrast_stack_i = contrast_stack[i]
#         edge_name_i = edge_name[i]

#         # Scale attention
#         # TODO: Remove once heads are balanced
#         contrast_stack_i = contrast_stack_i / np.nan_to_num(contrast_stack_i).max(axis=1).reshape((-1, 1))

#         # Format
#         df = pd.DataFrame(contrast_stack_i, index=pd.Series(columns), columns=contrast_subject_ids)
#         df = df.reset_index(names='Head').melt(id_vars='Head', var_name='Subject', value_name=edge_name_i).dropna()  # Melt
#         df = df.set_index('Subject').join(meta.set_index('SubID')[[target, target_comparison]]).reset_index()  # Join meta

#         # Filter to target heads
#         df = df.loc[df['Head'].apply(lambda s: s in (column, comparison))]

#         # Main target
#         p1 = sns.violinplot(data=df, x='Head', y=edge_name_i, hue=target, ax=axs[i])
#         p1.legend(title=target, bbox_to_anchor=(1.1, 1.05))
#         p1.set(xlabel=None, xticklabels=[])

#         # Comparison target
#         p2 = sns.violinplot(data=df, x='Head', y=edge_name_i, hue=target_comparison, ax=axs[i+3])
#         p2.legend(title=target_comparison, bbox_to_anchor=(1.1, 1.05))
#         plt.sca(p2)
#         plt.xticks(rotation=60)

#         # Get correlation p-values for targets (which must be numeric)
#         for j, tar in enumerate((target, target_comparison)):
#             for k, c in enumerate(np.unique(df['Head'])):
#                 try:
#                     pval = pearsonr(df.loc[df['Head']==c, edge_name_i], df.loc[df['Head']==c, tar])[1]
#                     axs[i+3*j].text(k, axs[i+3*j].get_ylim()[0] - (.15 if not j else .3), f'p={pval:.1e}', ha='center', va='center')
#                 except: continue

#     fig.savefig(f'../plots/group_differential_expression_{contrast_name}_{column}_{comparison}_{target}_{target_comparison}.pdf', format='pdf', transparent=True, backend='cairo')


### 4CD Linkage Cluster Enrichment

In [20]:
# for contrast_name, _, column, _, target, target_comparison in contrast_groupings:
#     for tar in (target, target_comparison):
#         # Get subject ids
#         group = None  # contrast_group
#         if group is None:
#             # Population
#             contrast_subjects = sum([v for k, v in get_contrast(contrast_name).items()], [])
#         else:
#             # Group
#             contrast_subjects = get_contrast(contrast_name)[group]

#         # Modify stack to include only contrast
#         df = np.nan_to_num(attention_stack[:, np.argwhere(np.array(columns)==column)[0][0], [s in contrast_subjects for s in subject_ids]])
#         new_subject_ids = [s for s in subject_ids if s in contrast_subjects]
#         df = pd.DataFrame(df, index=all_edges, columns=new_subject_ids)

#         # Get 100 most variant edges
#         df = df.iloc[df.to_numpy().var(axis=1).argsort()[::-1][:100]]

#         # Cluster
#         labels = KMeans(n_clusters=10, n_init=10).fit_predict(df.to_numpy().T)
#         labels += 1

#         # Get phenotypes
#         pheno = [meta.iloc[np.argwhere(meta['SubID'] == sid)[0][0]][tar] for sid in new_subject_ids]

#         # Format results
#         df = pd.DataFrame({'Cluster': labels, tar: pheno}, index=new_subject_ids)
#         df['count'] = 1
#         df = df.pivot_table(index='Cluster', columns=tar, values='count', aggfunc='sum').fillna(0)

#         # Transform to hypergeometric
#         df_np = df.to_numpy()
#         df_np_new = np.zeros_like(df_np)
#         for i, j in product(*[range(k) for k in df.shape]):
#             # i - cluster, j - target
#             dist = hypergeom(df_np.sum(), df_np[:, j].sum(), df_np[i, :].sum())
#             # Calculate probability of overrepresentation
#             df_np_new[i, j] = 1 - dist.cdf(df_np[i, j])
#         with np.errstate(divide='ignore'):
#             df_np_new = -np.log10(df_np_new)
#             df_np_new[np.isinf(df_np_new)] = np.nan
#         df = pd.DataFrame(df_np_new, index=df.index, columns=df.columns)

#         # Plot
#         fig, axs = get_mosaic([list(range(1))], scale=9)
#         sns.heatmap(df, vmin=0, vmax=3, cmap='rocket_r', cbar_kws={'label': '-log10(p)'}, ax=axs[0])
#         # plt.tight_layout()
#         fig.savefig(f'../plots/group_linkage_cluster_{contrast_name}_{column}_{tar}.pdf', format='pdf', transparent=True, backend='cairo')


### 4E Aggregate Graph Enrichment (MANUAL)

In [21]:
# # NOTE: Only top 100 are taken for aggregate due to memory concerns
# for contrast_name, group, column, _, _, _ in contrast_groupings:
#       # Load contrast
#       np.random.seed(42)
#       contrast_subjects = get_contrast(contrast_name)
#       gs = {
#             gname: concatenate_graphs(*[
#                   compute_graph(g)
#                   for g in load_many_graphs(np.random.choice(sids, 100, replace=False))[0]
#             ])
#             for gname, sids in contrast_subjects.items()
#       }

#       # Split into groups
#       # TODO: Make more general, perhaps add comparison group to arguments
#       g1_name = group
#       g1 = gs[g1_name]
#       g2_name = 'Control'
#       g2 = gs[g2_name]

#       # Get unique TFs
#       df = compare_graphs_enrichment(
#             g1,
#             g2,
#             sid_1=g1_name,
#             sid_2=g2_name,
#             nodes=synthetic_nodes_of_interest,
#             threshold=.01)

#       # Save to file
#       df.to_csv(f'../plots/genes_{contrast_name}_{group}_{column}.csv', index=False)


In [22]:
# # Enrichment
# for contrast_name, group, column, _, _, _ in contrast_groupings:
#     # MANUAL PROCESSING
#     # Run the output from above on Metascape as multiple gene list and perform
#     # enrichment.  From the all-in-one ZIP file, save the file from
#     # Enrichment_QC/GO_DisGeNET as '../plot/disgenet_{subject_id_1}_{subject_id_2}_{column}.csv' and
#     # Overlap_circos/CircosOverlapByGene.svg as '../plot/overlap_{subject_id_1}_{subject_id_2}_{column}.svg'

#     # Get enrichment
#     enrichment_file = f'../plots/disgenet_{contrast_name}_{group}_{column}.csv'
#     if enrichment_file is None: continue
#     enrichment = pd.read_csv(enrichment_file)

#     # Format
#     enrichment = format_enrichment(enrichment)

#     # Plot
#     fig, axs = get_mosaic([[0]*2], scale=9)
#     pl = sns.scatterplot(
#         enrichment,
#         x='Gene Set', y='Description',
#         size='-log10(p)',
#         color='black',
#         ax=axs[0])
#     # Formatting
#     pl.grid()
#     plt.xticks(rotation=90)
#     pl.set_aspect('equal', 'box')
#     pl.legend(title='-log10(p)', bbox_to_anchor=(1.2, 1.05))
#     # Zoom X1
#     margin = .5
#     min_xlim, max_xlim = pl.get_xlim()
#     min_xlim -= margin; max_xlim += margin
#     pl.set(xlim=(min_xlim, max_xlim))
#     # Save
#     fig.savefig(f'../plots/group_enrichment_{contrast_name}_{group}_{column}.pdf', format='pdf', transparent=True, backend='cairo')
